In [1]:
import pandas as pd
import glob
import os
import numpy as np
import inspect
import sys
sys.path.append('..')
%reload_ext autoreload
from config_paths_and_vars import *

In [2]:
def get_session_id(path):
    return os.path.normpath(path).split("info")[0].split(os.sep)[-2]

def custom_sort(subject):
    """
    Custom sorting function based on subject identifiers and their numbers.
    Sorts identifiers starting with TRD before any others.

    Args:
        subject (str): A string representing the subject identifier.

    Returns:
        str: A modified string used for sorting the subject.

    Example:
        >>> custom_sort('TRD7T1234_moretext')
        '1234'
        >>> custom_sort('subject_021_text')
        '_text'
    """
    if subject.startswith("TRD"):
        return subject.split("7T")[1][:4]
    else:
        return "_" + subject.split("021_", maxsplit = 1)[1][1:]
    
def print_modality_conditions(lambdafunc):
    if isinstance(lambdafunc, dict):
        for key, func in lambdafunc.items():
            try:
                source = inspect.getsource(func).strip()
            except OSError:
                source = "<lambda source unavailable>"
            print(f'"{key}": {source}')
    elif callable(lambdafunc):
        source = inspect.getsource(lambdafunc).strip()
        print(source)
    else:
        raise ValueError("You must paste either a lambda fuction or a dictionary of lambdafunctions.")

## Sequence Info overview
This script is dedicated to giving an overview which sequences were run during the scanning session of a group of scans (e.g. for all TGA participants). <br>
Make sure all participants have a dicominfo tsv file (you can create those by referring to the heudiconv_wrapper.ipynb in this directory). <br>
So it must be run AFTER the heudiconv_wrapper script. <br>

Requirements:
- *dicominfo tsv files* - for each session that should be considered, as generated by heudiconv. See heudiconv_wrapper notebook in this directory. 
- *config vars* as specified in config_paths_and_vars.py (will automatically be imported), e.g.
    - sorting rules - to classify scans (modality conditions).
    - path template - to search for spectroscopy scans (as these are not captured by heudiconv).
    - details about classifying fmri data into blocks

Output:
- *session_sequence_summary.tsv* - a tsv file that indicates which sequences were run for each session
- *sequence_type_count.tsv* - a tsv file that lists all sequences run count across participants

In [3]:
OUTPUT_DIR = os.path.join(os.getcwd(), "data")
HEUDICONV_INFO_ROOT = OUTPUT_DIR
SUMMARY_FILENAME = fr'{IDENTIFIER}_scan_sessions_sequence_summary.tsv'
COUNT_FILENAME = fr'{IDENTIFIER}_scan_sessions_sequence_type_count.tsv'

print(f"OUTPUT FILES to '{OUTPUT_DIR}':\n- {SUMMARY_FILENAME}\n- {COUNT_FILENAME}")

print_config_vars(["IDENTIFIER", "DICOMINFO_TSV_NAME", "SPECTRO_TEMPLATE", "NUM_TRS_ONE_FMRI_BLOCK", "FMRI_BLOCK_THRESHOLD"])
print("MODALITY_CONDITIONS=")
print_modality_conditions(get_config_vars()["MODALITY_CONDITIONS"])

OUTPUT FILES to '/home/luisass/Work/drive_backup_git/MR-analysis/MR-Sequence-inspection/scripts/data':
- PROJECT1_scan_sessions_sequence_summary.tsv
- PROJECT1_scan_sessions_sequence_type_count.tsv

=== CONFIG VARIABLES ============================================================
=== see /home/luisass/Work/drive_backup_git/MR-analysis/MR-Sequence-inspection/config_paths_and_vars.py
IDENTIFIER                = PROJECT1
DICOMINFO_TSV_NAME        = dicominfo_mod.tsv
SPECTRO_TEMPLATE          = /home/luisass/ishere/*/*/MRS-data
NUM_TRS_ONE_FMRI_BLOCK    = 418
FMRI_BLOCK_THRESHOLD      = 0.4
MODALITY_CONDITIONS=
"T1w": "T1w": lambda x: "t1" in x and "UNI" in x and "0.75" in x,
"T2w": "T2w": lambda x: "t2" in x,
"fMRI": "fMRI": lambda x: "face" in x and "ap" in x and not "SBRef" in x,
"DWI": "DWI": lambda x: "resolve_COR" in x,
"DTI": "DTI": lambda x: "dti" in x


In [4]:
#create output path if it doesn't exist
if not os.path.exists(OUTPUT_DIR):
    os.path.makedirs(OUTPUT_DIR, exist_ok = True)
    print(f"Created {OUTPUT_DIR}")

In [5]:
tsv_file_paths = glob.glob(fr"**/{DICOMINFO_TSV_NAME}", root_dir= HEUDICONV_INFO_ROOT, recursive=True, include_hidden=True)
tsv_file_paths = [os.path.join(HEUDICONV_INFO_ROOT, p) for p in tsv_file_paths]

#exclude scan IDs if specified
tsv_file_paths = [p for p in tsv_file_paths if not any(exclusion_ID in p for exclusion_ID in EXCLUDE_SCAN_IDS)]

print(f"{len(tsv_file_paths)} sessions found in total, with root '{HEUDICONV_INFO_ROOT}'.")

spectro_paths = glob.glob(SPECTRO_TEMPLATE)
IDs_w_spectro = [os.path.basename(os.path.dirname(os.path.dirname(p))) for p in spectro_paths]
print(f"\n{len(IDs_w_spectro)} sessions with spectroscopy scan, with template {SPECTRO_TEMPLATE}.")

0 sessions found in total, with root '/home/luisass/Work/drive_backup_git/MR-analysis/MR-Sequence-inspection/scripts/data'.

0 sessions with spectroscopy scan, with template /home/luisass/ishere/*/*/MRS-data.


In [6]:
session_IDs = [get_session_id(p) for p in tsv_file_paths]
session_IDs = sorted(session_IDs, key=custom_sort)
print("Sessions found:\n" + "\n".join(session_IDs))

Sessions found:



In [7]:
#prepare summary dataframe
sequence_overview_df = pd.DataFrame(0, index=range(len(session_IDs)), columns=["Session_ID", "T1w", "T2w", "fMRI", "fMRI_blocks","fMRI_num_TRs", "DWI", "DTI", "Spectro"])
sequence_overview_df["fMRI_blocks"] = sequence_overview_df["fMRI_blocks"].astype(float)
sequence_overview_df["fMRI_num_TRs"] = sequence_overview_df["fMRI_num_TRs"].astype(str) #because we are gonna have it as a list in the dataframe
sequence_overview_df["Session_ID"] = session_IDs

In [8]:
#collect all occuring series descriptions
all_series_descriptions = list()
#iterate through found dicom info tsvs
for tsv_file_path in tsv_file_paths:
    tsv_file = pd.read_csv(tsv_file_path, sep='\t', encoding='latin-1', on_bad_lines='skip')
    series_description = tsv_file["series_description"].values
    all_series_descriptions.extend(series_description)
    #session id and index in the overview df
    cur_id = get_session_id(tsv_file_path)
    sess_ID_index = sequence_overview_df[sequence_overview_df["Session_ID"] == cur_id].index
    #sometimes we have more than one fMRI run, so make a list
    fMRI_run_TRs = list()
    #now check all registered sequences in the current tsv file
    for index, row in tsv_file.iterrows():
        cur_series = row["series_description"]
        #if not null
        if cur_series == cur_series:
            #check for modalities
            for modality, condition in MODALITY_CONDITIONS.items():
                if condition(cur_series):
                    sequence_overview_df.loc[sess_ID_index, modality] = 1
                    break #exclusive

            if MODALITY_CONDITIONS["fMRI"](cur_series):
                num_TRs = tsv_file.loc[index, "dim4"]
                fMRI_run_TRs.append(num_TRs)

    #how many fMRI blocks?           
    if fMRI_run_TRs:          
        if any(fMRI_run_TRs > FMRI_BLOCK_THRESHOLD * num_TRs):
            num_blocks = 0.0
            for num_TRs in fMRI_run_TRs:
                if num_TRs / NUM_TRS_ONE_FMRI_BLOCK >= FMRI_BLOCK_THRESHOLD:
                    num_blocks = num_blocks + np.round(num_TRs / NUM_TRS_ONE_FMRI_BLOCK, 1)
            sequence_overview_df.loc[sess_ID_index, "fMRI_blocks"] = num_blocks
        sequence_overview_df.loc[sess_ID_index, "fMRI_num_TRs"] = str(", ".join([str(int(tr)) for tr in fMRI_run_TRs]))

    if cur_id in IDs_w_spectro:
        sequence_overview_df.loc[sess_ID_index, "Spectro"] = 1
    

sequence_overview_df

,Session_ID,T1w,T2w,fMRI,fMRI_blocks,fMRI_num_TRs,DWI,DTI,Spectro


In [9]:
description_counts = pd.Series(all_series_descriptions).value_counts()
description_counts = description_counts.reset_index()
description_counts.columns = ["series_description", "count"]
description_counts[:30]

,series_description,count


## Write to system

summary file

In [10]:
def are_dfs_different(df_1, df_2):
    if df_1.shape != df_2.shape:
        return True
    
    return not df_1.equals(df_2)

In [11]:
# ==============================================
UPDATE = True #should existing file be updated? 
# ==============================================

sequence_summary_path = os.path.abspath(os.path.join(OUTPUT_DIR, SUMMARY_FILENAME))

#create new file if not existing
if not os.path.exists(sequence_summary_path):
    updated_df = sequence_overview_df.copy()
    updated_df["Comments"] = ""
    updated_df.to_csv(sequence_summary_path, sep = "\t", index = False)
    print(f"Created file at '{sequence_summary_path}'")

#or possibly update existing one
else:

    old_sequence_overview_df = pd.read_csv(sequence_summary_path, sep = "\t")
    cols_without_comments = list(old_sequence_overview_df.columns)
    cols_without_comments.remove("Comments")

    if are_dfs_different(sequence_overview_df, old_sequence_overview_df[cols_without_comments]):
        print("Changes detected. Attempting update...")
        if UPDATE:
            #update the file
            comment_column = old_sequence_overview_df["Comments"]
            updated_df = sequence_overview_df.copy()
            updated_df["Comments"] = comment_column
            os.remove(sequence_summary_path)
            updated_df.to_csv(sequence_summary_path, sep = "\t", index = False)
            print(f"Updated table at '{sequence_summary_path}'")
        else:
            raise ValueError(f"File '{sequence_summary_path}' exists. Change output path or flag to update to replace existing file.")
    else:
            #leave as is
            print("No changes detected. Updating not necessary.")
   
    
#If you get the error "the process cannot access the file", you probably have it open in some editor :) please close it so it can be overwritten


Changes detected. Attempting update...
Updated table at '/home/luisass/Work/drive_backup_git/MR-analysis/MR-Sequence-inspection/scripts/data/PROJECT1_scan_sessions_sequence_summary.tsv'


count file

In [12]:
# ==============================================
OVERWRITE = True #should existing file be overwritten?
# ==============================================

description_count_filepath = os.path.abspath(os.path.join(OUTPUT_DIR, COUNT_FILENAME))
if not os.path.exists(description_count_filepath) or OVERWRITE:
    description_counts.to_csv(description_count_filepath, sep = "\t", index = False)
    print(f"Created or overwrote file at '{description_count_filepath}'")
else:
    raise ValueError(f"File '{description_count_filepath}' exists. Change output path or flag to overwrite to replace existing file.")


Created or overwrote file at '/home/luisass/Work/drive_backup_git/MR-analysis/MR-Sequence-inspection/scripts/data/PROJECT1_scan_sessions_sequence_type_count.tsv'


modality rules

In [13]:
# ==============================================
OVERWRITE = True #should existing file be overwritten?
MODALITY_RULES_FILENAME = fr'{IDENTIFIER}_scan_sessions_modality_rules.txt'
# ==============================================

modality_rules_to_text = list()
modality_rules_to_text.append("Rules used to determine if a scan exists for a certain modality and session.\
                              \nThe 'series_description' field in the dicom header is used as reference.\n")
for v in MODALITY_CONDITIONS.values():
    info = "".join(inspect.getsourcelines(v)[0][0].split("\"")[1:])
    info = info.replace(",\n","")
    modality_rules_to_text.append(info)


rule_filepath = os.path.abspath(os.path.join(OUTPUT_DIR, MODALITY_RULES_FILENAME))
if not os.path.exists(rule_filepath) or OVERWRITE:
    with open(rule_filepath, 'w') as f:
        for line in modality_rules_to_text:
            f.write(f"{line}\n")
    print(f"Created or overwrote file at '{rule_filepath}'")
else:
    raise ValueError(f"File '{rule_filepath}' exists. Change output path or flag to overwrite to replace existing file.")


Created or overwrote file at '/home/luisass/Work/drive_backup_git/MR-analysis/MR-Sequence-inspection/scripts/data/PROJECT1_scan_sessions_modality_rules.txt'
